# NBA Machine Learning

The objective for our project will be to create a deep learning model that will be able to predict future outcomes of a basketball season. 
This is done because this could be used as a viable tool in the process of sports betting. The model would function almost exactly the same as one that predicts stock price and signals when to buy or sell, the only difference is that it would predict a player's stat and would signal when to bet instead of buying/selling.

Our project will use historical data from NBA teams such as previous season's records, shot making percentages, defensive ratings, 
and several others from the last ten seasons in a deep learning model to predict the next season's standings for each season and individual player statistics.


In [1]:
#import libraries required to run program
import numpy as np
import pandas as pd
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder

2023-06-06 19:34:59.614717: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#import csv containing Jokic's statistics
jokic_csv_path = Path("Resources/jokic_ultimate.csv")
jokic_df = pd.read_csv(jokic_csv_path)

In [3]:
#dropped columns that were not relevant to model
jokic_df.drop(columns= ["Rk","Tm","Date","Age","Opp"], axis=1, inplace = True)
jokic_df.head()
jokic_df.columns

Index(['G', 'Unnamed: 5', 'Unnamed: 7', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL',
       'BLK', 'TOV', 'PF', 'PTS', 'GmSc', '+/-', 'TS%', 'eFG%', 'ORB%', 'DRB%',
       'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg', 'BPM'],
      dtype='object')

In [4]:
#added titles to columns without names
jokic_df.rename(columns={'Unnamed: 5':'Home_or_Away'},inplace= True)
jokic_df.rename(columns={'Unnamed: 7':'MOV'},inplace= True)

In [5]:
#changed Home or Away column so that if the game was at home it is a value of 0 and if away it is 1
jokic_df['Home_or_Away'] = jokic_df['Home_or_Away'].replace('@', 1)
jokic_df = jokic_df.fillna(0)
jokic_df

,G,Home_or_Away,MOV,GS,MP,FG,FGA,FG%,3P,3PA,...,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,BPM
0,1.0,1.0,L (-21),1,33:25,12.0,17.0,0.706,1.0,3.0,...,8.0,7.4,37.9,4.3,0.0,14.4,26.8,136.0,119.0,13.7
1,1.0,1.0,L (-21),1,33:25,12.0,17.0,0.706,1.0,3.0,...,31.6,25.6,32.7,1.8,0.0,7.0,22.5,124.0,83.0,6.4
2,1.0,0.0,W (+29),1,28:29,6.0,12.0,0.500,0.0,0.0,...,8.0,7.4,37.9,4.3,0.0,14.4,26.8,136.0,119.0,13.7
3,1.0,0.0,W (+29),1,28:29,6.0,12.0,0.500,0.0,0.0,...,31.6,25.6,32.7,1.8,0.0,7.0,22.5,124.0,83.0,6.4
4,2.0,1.0,W (+5),1,34:24,7.0,13.0,0.538,1.0,2.0,...,34.9,22.0,38.5,0.0,0.0,21.9,27.2,138.0,119.0,7.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
578,77.0,0.0,W (+28),1,28:40,9.0,10.0,0.900,0.0,1.0,...,36.8,21.7,47.2,0.0,0.0,30.6,25.7,125.0,87.0,11.9
579,78.0,0.0,W (+9),1,36:45,9.0,15.0,0.600,1.0,4.0,...,33.5,21.5,37.6,2.8,2.3,10.7,23.2,139.0,111.0,12.8
580,0.0,1.0,L (-7),Inactive,0,0.0,0.0,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581,79.0,1.0,L (-10),1,16:03,1.0,6.0,0.167,0.0,0.0,...,24.2,18.9,15.7,3.0,5.9,12.7,22.5,57.0,112.0,-15.1


In [6]:
#remove rows with games Jokic did not play
jokic_df = jokic_df[jokic_df['GS'] != "Inactive"]
jokic_df = jokic_df[jokic_df['GS'] != "Did Not Dress"]
jokic_df = jokic_df[jokic_df['GS'] != "Did Not Play"]
display(jokic_df)

,G,Home_or_Away,MOV,GS,MP,FG,FGA,FG%,3P,3PA,...,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,BPM
0,1.0,1.0,L (-21),1,33:25,12.0,17.0,0.706,1.0,3.0,...,8.0,7.4,37.9,4.3,0.0,14.4,26.8,136.0,119.0,13.7
1,1.0,1.0,L (-21),1,33:25,12.0,17.0,0.706,1.0,3.0,...,31.6,25.6,32.7,1.8,0.0,7.0,22.5,124.0,83.0,6.4
2,1.0,0.0,W (+29),1,28:29,6.0,12.0,0.500,0.0,0.0,...,8.0,7.4,37.9,4.3,0.0,14.4,26.8,136.0,119.0,13.7
3,1.0,0.0,W (+29),1,28:29,6.0,12.0,0.500,0.0,0.0,...,31.6,25.6,32.7,1.8,0.0,7.0,22.5,124.0,83.0,6.4
4,2.0,1.0,W (+5),1,34:24,7.0,13.0,0.538,1.0,2.0,...,34.9,22.0,38.5,0.0,0.0,21.9,27.2,138.0,119.0,7.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
577,76.0,1.0,L (-14),1,27:42,4.0,10.0,0.400,0.0,3.0,...,12.7,9.1,35.8,6.5,0.0,34.6,24.3,73.0,104.0,-1.5
578,77.0,0.0,W (+28),1,28:40,9.0,10.0,0.900,0.0,1.0,...,36.8,21.7,47.2,0.0,0.0,30.6,25.7,125.0,87.0,11.9
579,78.0,0.0,W (+9),1,36:45,9.0,15.0,0.600,1.0,4.0,...,33.5,21.5,37.6,2.8,2.3,10.7,23.2,139.0,111.0,12.8
581,79.0,1.0,L (-10),1,16:03,1.0,6.0,0.167,0.0,0.0,...,24.2,18.9,15.7,3.0,5.9,12.7,22.5,57.0,112.0,-15.1


In [7]:
#convert minutes played column to seconds played 
def convert_to_numerical(time_obj):
    minutes, seconds = time_obj.split(':')
    total_seconds = int(minutes) * 60 + int(seconds)
    return total_seconds

jokic_df['MP'] = jokic_df['MP'].apply(convert_to_numerical)
jokic_df.rename(columns={'MP':'Seconds_played'},inplace =True)
jokic_df.dtypes

G                 float64
Home_or_Away      float64
MOV                object
GS                 object
Seconds_played      int64
FG                float64
FGA               float64
FG%               float64
3P                float64
3PA               float64
3P%               float64
FT                float64
FTA               float64
FT%               float64
ORB               float64
DRB               float64
TRB               float64
AST               float64
STL               float64
BLK               float64
TOV               float64
PF                float64
PTS               float64
GmSc              float64
+/-               float64
TS%               float64
eFG%              float64
ORB%              float64
DRB%              float64
TRB%              float64
AST%              float64
STL%              float64
BLK%              float64
TOV%              float64
USG%              float64
ORtg              float64
DRtg              float64
BPM               float64
dtype: objec

In [8]:
jokic_df.columns

Index(['G', 'Home_or_Away', 'MOV', 'GS', 'Seconds_played', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST',
       'STL', 'BLK', 'TOV', 'PF', 'PTS', 'GmSc', '+/-', 'TS%', 'eFG%', 'ORB%',
       'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg',
       'BPM'],
      dtype='object')

In [9]:
#converted columns with float64 types to int64 types
jokic_int_df = jokic_df[['G', 'Home_or_Away','FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST',
       'STL', 'BLK', 'TOV', 'PF', 'PTS', 'GmSc', '+/-', 'TS%', 'eFG%', 'ORB%',
       'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg',
       'BPM']].round().astype('int64')


In [10]:
#concat dataframes that contained all columns
jokic_df.drop(columns= ['G', 'Home_or_Away','FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST',
       'STL', 'BLK', 'TOV', 'PF', 'PTS', 'GmSc', '+/-', 'TS%', 'eFG%', 'ORB%',
       'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg',
       'BPM'], axis=1, inplace = True)
jokic_full_df = pd.concat([jokic_df, jokic_int_df], axis=1)
jokic_full_df.tail()

,MOV,GS,Seconds_played,G,Home_or_Away,FG,FGA,FG%,3P,3PA,...,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,BPM
577,L (-14),1,1662,76,1,4,10,0,0,3,...,13,9,36,6,0,35,24,73,104,-2
578,W (+28),1,1720,77,0,9,10,1,0,1,...,37,22,47,0,0,31,26,125,87,12
579,W (+9),1,2205,78,0,9,15,1,1,4,...,34,22,38,3,2,11,23,139,111,13
581,L (-10),1,963,79,1,1,6,0,0,0,...,24,19,16,3,6,13,22,57,112,-15
582,W (+4),1,2195,80,0,13,26,0,1,7,...,30,20,12,4,0,13,38,100,93,4


In [11]:
#changed MOV to win or loss column and another with actual margin
jokic_full_df['WinOrLoss'] = jokic_full_df['MOV'].str[0].map({'L':0,'W':1})
jokic_full_df['Margin'] = jokic_full_df['MOV'].str[1:].str.extract("(\d+)").astype(int)
jokic_full_df.dtypes

MOV               object
GS                object
Seconds_played     int64
G                  int64
Home_or_Away       int64
FG                 int64
FGA                int64
FG%                int64
3P                 int64
3PA                int64
3P%                int64
FT                 int64
FTA                int64
FT%                int64
ORB                int64
DRB                int64
TRB                int64
AST                int64
STL                int64
BLK                int64
TOV                int64
PF                 int64
PTS                int64
GmSc               int64
+/-                int64
TS%                int64
eFG%               int64
ORB%               int64
DRB%               int64
TRB%               int64
AST%               int64
STL%               int64
BLK%               int64
TOV%               int64
USG%               int64
ORtg               int64
DRtg               int64
BPM                int64
WinOrLoss          int64
Margin             int64


In [12]:
#dropped MOV, G and GS
jokic_full_df.drop(columns= ["MOV","GS","G"], axis=1, inplace = True)

In [13]:
display(jokic_full_df)

,Seconds_played,Home_or_Away,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,...,AST%,STL%,BLK%,TOV%,USG%,ORtg,DRtg,BPM,WinOrLoss,Margin
0,2005,1,12,17,1,1,3,0,2,2,...,38,4,0,14,27,136,119,14,0,21
1,2005,1,12,17,1,1,3,0,2,2,...,33,2,0,7,22,124,83,6,0,21
2,1709,0,6,12,0,0,0,0,1,3,...,38,4,0,14,27,136,119,14,1,29
3,1709,0,6,12,0,0,0,0,1,3,...,33,2,0,7,22,124,83,6,1,29
4,2064,1,7,13,1,1,2,0,11,11,...,38,0,0,22,27,138,119,7,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
577,1662,1,4,10,0,0,3,0,2,3,...,36,6,0,35,24,73,104,-2,0,14
578,1720,0,9,10,1,0,1,0,2,3,...,47,0,0,31,26,125,87,12,1,28
579,2205,0,9,15,1,1,4,0,3,4,...,38,3,2,11,23,139,111,13,1,9
581,963,1,1,6,0,0,0,0,0,2,...,16,3,6,13,22,57,112,-15,0,10


In [14]:
jokic_full_df.to_csv('final_stats_jokic.csv', index=False)

In [15]:
y = jokic_full_df['PTS'].values
display(y)

array([27, 27, 13, 13, 26, 26, 27, 27, 19, 19, 20, 20,  9,  9, 43, 43, 31,
       31, 28, 28, 12, 12, 24, 24, 23, 23, 39, 39, 15, 15, 30, 30, 21, 21,
       53, 53, 26, 26, 29, 29, 24, 24, 32, 32, 29, 29, 34, 34,  8,  8, 23,
       23, 24, 24, 30, 30, 27, 27, 31, 39, 19, 32, 17, 24, 32, 19, 33, 31,
       43, 25, 40, 13, 29, 41, 20, 40, 19, 30, 24, 12, 28, 14, 21, 17, 36,
       31, 25, 24, 26, 22, 14, 20, 29, 30, 27, 14, 24, 15, 40, 14, 18, 17,
       18, 37, 35, 28, 30, 24, 22, 31, 31, 25, 14,  6, 27, 27, 25, 25, 32,
       32, 26, 26, 24, 24, 37, 37, 24, 24, 37, 37, 11, 11, 30, 30, 26, 23,
       34, 28, 25, 22, 28, 35, 30, 24, 18, 32, 17, 39, 22, 35, 28, 27, 20,
       13, 29, 26, 22, 24, 27, 26, 33, 22, 21, 20, 17, 25, 49, 26, 34, 28,
       26, 29, 18, 21, 25, 27, 21, 23, 28, 26, 35, 25, 18,  8, 22, 46, 32,
       38, 23, 26, 22, 29, 32, 23, 30, 28, 35, 26, 37, 38, 38, 41, 35, 29,
       29, 34, 34, 24, 24, 38, 38, 19, 19, 36, 36, 26, 26, 16, 16, 17, 17,
       38, 38, 19, 19, 36

In [16]:
X = jokic_full_df.drop(columns= 'PTS').values
display(X)

array([[2005,    1,   12, ...,   14,    0,   21],
       [2005,    1,   12, ...,    6,    0,   21],
       [1709,    0,    6, ...,   14,    1,   29],
       ...,
       [2205,    0,    9, ...,   13,    1,    9],
       [ 963,    1,    1, ...,  -15,    0,   10],
       [2195,    0,   13, ...,    4,    1,    4]])

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [18]:
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [19]:
number_input_features = 36

In [20]:
number_output_neurons = 1

In [21]:
hidden_nodes_layer1 = 16

In [22]:
hidden_nodes_layer2 = 8

In [23]:
hidden_nodes_layer3 = 4

In [24]:
nn = Sequential()

In [25]:
nn.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

In [26]:
nn.add(Dense(units=hidden_nodes_layer2, activation="relu"))

In [27]:
nn.add(Dense(units=hidden_nodes_layer3, activation="relu"))

In [28]:
nn.add(Dense(units=1, activation="linear"))

In [29]:
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                592       
                                                                 
 dense_1 (Dense)             (None, 8)                 136       
                                                                 
 dense_2 (Dense)             (None, 4)                 36        
                                                                 
 dense_3 (Dense)             (None, 1)                 5         
                                                                 
Total params: 769
Trainable params: 769
Non-trainable params: 0
_________________________________________________________________


In [30]:
nn.compile(loss= "mse", optimizer="adam", metrics=["mse"])

In [31]:
jokic_deep_model = nn.fit(X_train_scaled, y_train, epochs=1000)

Epoch 1/1000
14/14 [==============================] - 1s 5ms/step - loss: 637.5964 - mse: 637.5964
Epoch 2/1000
14/14 [==============================] - 0s 3ms/step - loss: 626.9054 - mse: 626.9054
Epoch 3/1000
14/14 [==============================] - 0s 2ms/step - loss: 614.0994 - mse: 614.0994
Epoch 4/1000
14/14 [==============================] - 0s 2ms/step - loss: 597.9686 - mse: 597.9686
Epoch 5/1000
14/14 [==============================] - 0s 2ms/step - loss: 578.4214 - mse: 578.4214
Epoch 6/1000
14/14 [==============================] - 0s 2ms/step - loss: 554.2271 - mse: 554.2271
Epoch 7/1000
14/14 [==============================] - 0s 2ms/step - loss: 523.5458 - mse: 523.5458
Epoch 8/1000
14/14 [==============================] - 0s 2ms/step - loss: 483.2499 - mse: 483.2499
Epoch 9/1000
14/14 [==============================] - 0s 2ms/step - loss: 431.9117 - mse: 431.9117
Epoch 10/1000
14/14 [==============================] - 0s 2ms/step - loss: 367.7665 - mse: 367.7665
Epoch 11/

In [32]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)

5/5 - 0s - loss: 2.2233 - mse: 2.2233 - 170ms/epoch - 34ms/step


In [33]:
predictions = (nn.predict(X_test_scaled)).astype("int32")

5/5 [==============================] - 0s 4ms/step


In [34]:
results = pd.DataFrame({"predictions": predictions.ravel(), "actual":y_test})

In [50]:
display(X_test_scaled)

array([[ 0.15775604, -1.00240096,  1.07763135, ...,  0.78162631,
        -1.34114045, -0.78110197],
       [-0.14088936, -1.00240096, -0.89577261, ..., -0.07675112,
        -1.34114045, -0.17228044],
       [ 2.29703223, -1.00240096,  1.3595462 , ...,  1.026877  ,
         0.74563406, -0.53757336],
       ...,
       [-1.00939893,  0.99760479, -0.61385776, ...,  2.13050512,
         0.74563406,  2.99359156],
       [-0.43039255, -1.00240096, -0.89577261, ..., -0.19937646,
         0.74563406, -0.17228044],
       [ 0.7215254 , -1.00240096,  1.3595462 , ...,  4.70563741,
         0.74563406, -0.78110197]])

In [35]:
display(results)

,predictions,actual
0,28,29
1,15,14
2,38,38
3,30,33
4,42,41
...,...,...
135,22,18
136,37,36
137,14,18
138,20,20


In [42]:
#save model
nn.save('jokic_deep_model.h5')

In [43]:
#Alternate Model for predicting Assists
y2 = jokic_full_df['AST'].values
X2 = jokic_full_df.drop(columns= 'AST').values
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, random_state=1)
scaler = StandardScaler()
X_scaler2 = scaler.fit(X_train2)
X_train_scaled2 = X_scaler2.transform(X_train2)
X_test_scaled2 = X_scaler2.transform(X_test2)
nn2 = Sequential()
nn2.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))
nn2.add(Dense(units=hidden_nodes_layer2, activation="relu"))
nn2.add(Dense(units=hidden_nodes_layer3, activation="relu"))
nn2.add(Dense(units=1, activation="linear"))
nn2.compile(loss= "mse", optimizer="adam", metrics=["mse"])
jokic_deep_model_assist = nn2.fit(X_train_scaled2, y_train2, epochs=1000)

Epoch 1/1000
14/14 [==============================] - 1s 3ms/step - loss: 59.9232 - mse: 59.9232
Epoch 2/1000
14/14 [==============================] - 0s 3ms/step - loss: 55.4887 - mse: 55.4887
Epoch 3/1000
14/14 [==============================] - 0s 2ms/step - loss: 50.9474 - mse: 50.9474
Epoch 4/1000
14/14 [==============================] - 0s 3ms/step - loss: 46.2160 - mse: 46.2160
Epoch 5/1000
14/14 [==============================] - 0s 3ms/step - loss: 41.2126 - mse: 41.2126
Epoch 6/1000
14/14 [==============================] - 0s 3ms/step - loss: 35.8857 - mse: 35.8857
Epoch 7/1000
14/14 [==============================] - 0s 2ms/step - loss: 30.4968 - mse: 30.4968
Epoch 8/1000
14/14 [==============================] - 0s 3ms/step - loss: 25.2515 - mse: 25.2515
Epoch 9/1000
14/14 [==============================] - 0s 3ms/step - loss: 20.8746 - mse: 20.8746
Epoch 10/1000
14/14 [==============================] - 0s 3ms/step - loss: 17.3522 - mse: 17.3522
Epoch 11/1000
14/14 [========

In [46]:
predictions2 = (nn2.predict(X_test_scaled2)).astype("int32")

5/5 [==============================] - 0s 5ms/step


In [47]:
model_loss2, model_accuracy2 = nn2.evaluate(X_test_scaled2, y_test2, verbose=2)

5/5 - 0s - loss: 3.9174 - mse: 3.9174 - 38ms/epoch - 8ms/step


In [48]:
results2 = pd.DataFrame({"predictions2": predictions2.ravel(), "actual":y_test2})

In [49]:
display(results2)

,predictions2,actual
0,8,10
1,5,7
2,11,9
3,6,7
4,3,4
...,...,...
135,11,8
136,6,5
137,14,15
138,7,6


In [51]:
opposing_team_df = pd.read_csv('Resources/revised_opposing.csv')

In [52]:
opposing_team_df['Seconds_played'] = opposing_team_df['MP'] * 60

In [53]:
opposing_team_df.set_index('Team', inplace=True)

In [54]:
opposing_team_df = opposing_team_df[['MP', 'FG%', '3P%', 'FT%', 'TS%', 'USG%','PTS', 'TRB',
       'AST', 'Seconds_played']].round().astype('int64')

In [55]:
opposing_team_df.drop(columns= ['MP'], axis=1, inplace = True)

In [56]:
display(opposing_team_df)

,FG%,3P%,FT%,TS%,USG%,PTS,TRB,AST,Seconds_played
Team,,,,,,,,,
Atlanta,0,0,1,1,26,18,11,6,1806
Boston,1,0,1,1,28,22,10,6,1848
Charlotte,1,0,1,1,29,22,14,7,1926
Chicago,1,0,1,1,27,18,10,8,1680
Cleveland,1,0,1,1,24,18,11,7,1872
Dallas,1,0,1,1,27,18,11,6,1800
Detroit,1,0,1,1,27,21,9,6,1830
Golden State,0,0,1,1,27,18,10,7,1776
Houston,1,0,1,1,29,19,10,6,1656


In [72]:
eastern_df = opposing_team_df.loc[['Washington', 'Atlanta', 'Milwaukee', 'Boston', 'Philadelphia', 'Cleveland', 'New York', 'New Jersey', 'Miami', 'Toronto', 'Chicago', 'Indiana', 'Orlando', 'Charlotte', 'Detroit']].copy()
eastern_df

,FG%,3P%,FT%,TS%,USG%,PTS,TRB,AST,Seconds_played
Team,,,,,,,,,
Washington,1,1,1,1,27,22,11,6,1842
Atlanta,0,0,1,1,26,18,11,6,1806
Milwaukee,1,0,1,1,27,21,10,8,1884
Boston,1,0,1,1,28,22,10,6,1848
Philadelphia,1,0,1,1,27,22,10,8,1980
Cleveland,1,0,1,1,24,18,11,7,1872
New York,1,0,1,1,29,22,10,6,1806
New Jersey,1,0,1,1,27,23,12,6,1818
Miami,1,0,1,1,25,20,11,6,1956


In [73]:
WX = eastern_df.drop(columns= 'PTS').values
display(WX)
Wy = eastern_df['PTS'].values
display(Wy)

array([[   1,    1,    1,    1,   27,   11,    6, 1842],
       [   0,    0,    1,    1,   26,   11,    6, 1806],
       [   1,    0,    1,    1,   27,   10,    8, 1884],
       [   1,    0,    1,    1,   28,   10,    6, 1848],
       [   1,    0,    1,    1,   27,   10,    8, 1980],
       [   1,    0,    1,    1,   24,   11,    7, 1872],
       [   1,    0,    1,    1,   29,   10,    6, 1806],
       [   1,    0,    1,    1,   27,   12,    6, 1818],
       [   1,    0,    1,    1,   25,   11,    6, 1956],
       [   1,    0,    1,    1,   25,   11,    6, 1776],
       [   1,    0,    1,    1,   27,   10,    8, 1680],
       [   1,    0,    1,    1,   31,   10,    5, 1890],
       [   0,    0,    1,    1,   24,   10,    8, 1914],
       [   1,    0,    1,    1,   29,   14,    7, 1926],
       [   1,    0,    1,    1,   27,    9,    6, 1830]])

array([22, 18, 21, 22, 22, 18, 22, 23, 20, 19, 18, 25, 18, 22, 21])

In [74]:
WX_train, WX_test, Wy_train, Wy_test = train_test_split(WX, Wy, random_state=1)

In [76]:
WX_scaler = scaler.fit(WX_train)
WX_train_scaled = scaler.transform(WX_train)
WX_test_scaled = WX_scaler.transform(WX_test)

In [77]:
future_game = (nn.predict(WX_test_scaled)).astype("int32")

2023-06-06 22:44:18.384403: I tensorflow/core/common_runtime/executor.cc:1197] [/job:localhost/replica:0/task:0/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: Matrix size-incompatible: In[0]: [4,8], In[1]: [36,16]
	 [[{{node sequential/dense/Relu}}]]


InvalidArgumentError: Graph execution error:

Detected at node 'sequential/dense/Relu' defined at (most recent call last):
    File "/Users/nahinhayat/opt/anaconda3/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Users/nahinhayat/opt/anaconda3/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/Users/nahinhayat/opt/anaconda3/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Users/nahinhayat/opt/anaconda3/lib/python3.9/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/Users/nahinhayat/opt/anaconda3/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/Users/nahinhayat/opt/anaconda3/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/Us